# Reconnaissance d'entités nommées avec SpaCy

La documentation est accessible ici: https://spacy.io/api

## Imports

In [1]:
from collections import defaultdict
import sys
import os
import spacy
from spacy.lang.fr.examples import sentences

In [2]:
nlp = spacy.load('fr_core_news_md')

## Exemple sur un corpus de test fourni par SpaCy

In [3]:
# Imprimer le corpus de Spacy
sentences

['Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars',
 "Les voitures autonomes déplacent la responsabilité de l'assurance vers les constructeurs",
 "San Francisco envisage d'interdire les robots coursiers sur les trottoirs",
 'Londres est une grande ville du Royaume-Uni',
 'L’Italie choisit ArcelorMittal pour reprendre la plus grande aciérie d’Europe',
 "Apple lance HomePod parce qu'il se sent menacé par l'Echo d'Amazon",
 "La France ne devrait pas manquer d'électricité cet été, même en cas de canicule",
 'Nouvelles attaques de Trump contre le maire de Londres',
 'Où es-tu ?',
 'Qui est le président de la France ?',
 'Où est la capitale des États-Unis ?',
 'Quand est né Barack Obama ?']

In [4]:
# Isoler la première phrase
sent = sentences[0]
sent

'Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars'

In [5]:
# Traiter la phrase avec Spacy
doc = nlp(sent)

In [6]:
type(doc)

spacy.tokens.doc.Doc

In [7]:
doc.text

'Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars'

In [8]:
doc.to_json()

{'text': 'Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars',
 'ents': [{'start': 0, 'end': 5, 'label': 'ORG'}],
 'sents': [{'start': 0, 'end': 72}],
 'tokens': [{'id': 0,
   'start': 0,
   'end': 5,
   'tag': 'PROPN',
   'pos': 'PROPN',
   'morph': '',
   'lemma': 'Apple',
   'dep': 'nsubj',
   'head': 1},
  {'id': 1,
   'start': 6,
   'end': 13,
   'tag': 'VERB',
   'pos': 'VERB',
   'morph': 'Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin',
   'lemma': 'cherche',
   'dep': 'ROOT',
   'head': 1},
  {'id': 2,
   'start': 14,
   'end': 15,
   'tag': 'ADP',
   'pos': 'ADP',
   'morph': '',
   'lemma': 'à',
   'dep': 'mark',
   'head': 3},
  {'id': 3,
   'start': 16,
   'end': 23,
   'tag': 'VERB',
   'pos': 'VERB',
   'morph': 'VerbForm=Inf',
   'lemma': 'acheter',
   'dep': 'xcomp',
   'head': 1},
  {'id': 4,
   'start': 24,
   'end': 27,
   'tag': 'DET',
   'pos': 'DET',
   'morph': 'Definite=Ind|Gender=Fem|Number=Sing|PronType=Art',
   'lemma': 'un',
   

In [9]:
# Appliquer le test sur toutes les phrases
for sent in sentences:
    doc = nlp(sent)
    entities = []
    for ent in doc.ents:
        entities.append(f"{ent.text} ({ent.label_})")
    if entities:
        print(f"'{doc.text}' contient les entités suivantes : {', '.join(entities)}")
    else:
        print(f"'{doc.text}' ne contient aucune entité")

'Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars' contient les entités suivantes : Apple (ORG)
'Les voitures autonomes déplacent la responsabilité de l'assurance vers les constructeurs' ne contient aucune entité
'San Francisco envisage d'interdire les robots coursiers sur les trottoirs' contient les entités suivantes : San Francisco (LOC)
'Londres est une grande ville du Royaume-Uni' contient les entités suivantes : Londres (LOC), Royaume-Uni (LOC)
'L’Italie choisit ArcelorMittal pour reprendre la plus grande aciérie d’Europe' contient les entités suivantes : Italie (LOC), ArcelorMittal (ORG), Europe (LOC)
'Apple lance HomePod parce qu'il se sent menacé par l'Echo d'Amazon' contient les entités suivantes : Apple (ORG), HomePod (ORG), Echo (ORG), Amazon (ORG)
'La France ne devrait pas manquer d'électricité cet été, même en cas de canicule' contient les entités suivantes : La France (LOC)
'Nouvelles attaques de Trump contre le maire de Londres' contient les entit

## Appliquer la reconnaissance d'entités nommées sur notre corpus

In [10]:
# Charger le texte
n=900000
year=1874
# Lister les fichiers de cette année
data_path = '../data'
txt_path = '../data/txt'
txts = [f for f in os.listdir(txt_path) if os.path.isfile(os.path.join(txt_path, f)) and str(year) in f and "RptAn" in f]
txts

['Lkn_1874_Tome_RptAn_Part_8.txt']

In [11]:
# Stocker le contenu de ces fichiers dans une liste
content_list = []
for txt in txts:
    with open(os.path.join(txt_path, txt), 'r') as f:
        content_list.append(f.read())

In [12]:
# Ecrire tout le contenu dans un fichier temporaire
temp_path = '../data/tmp'
if not os.path.exists(temp_path):
    os.mkdir(temp_path)
with open(os.path.join(temp_path, f'{year}.txt'), 'w') as f:
    f.write(' '.join(content_list))

In [13]:
with open(os.path.join(temp_path, f'{year}.txt'), 'r') as f:
    text = f.read()[:n]


In [14]:
%%time
# Traiter le texte

doc = nlp(text)

CPU times: user 7min 27s, sys: 5min 42s, total: 13min 9s
Wall time: 14min 55s


In [15]:
# Compter les entités
people = defaultdict(int)
organisation = defaultdict(int)
location = defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "PER" and len(ent.text) > 3:
        people[ent.text] += 1
    if ent.label_ == "ORG" and len(ent.text) > 3:
        organisation[ent.text] += 1
    if ent.label_ == "LOC" and len(ent.text) > 3:
        location[ent.text] += 1

In [16]:
# Trier et imprimer

sorted_people = sorted(people.items(), key=lambda kv: kv[1], reverse=True)
sorted_organisation = sorted(organisation.items(), key=lambda kv: kv[1], reverse=True)
sorted_location = sorted(location.items(), key=lambda kv: kv[1], reverse=True)

print("Personnes")
for person, freq in sorted_people[:50]:
    print(f"{person} apparait {freq} fois dans le corpus")
print("Organisations")
for org, freq in sorted_organisation[:50]:
    print(f"{org} apparait {freq} fois dans le corpus")
print("Locations")
for loc, freq in sorted_location[:50]:
    print(f"{loc} apparait {freq} fois dans le corpus")

Personnes
Messieurs apparait 58 fois dans le corpus
VAN DEN KERCKHOVE apparait 19 fois dans le corpus
Van Den Kerckhove apparait 11 fois dans le corpus
Bourgmestre apparait 11 fois dans le corpus
q u i apparait 10 fois dans le corpus
Echevins apparait 9 fois dans le corpus
Subside apparait 9 fois dans le corpus
Recettes apparait 8 fois dans le corpus
Siroul apparait 8 fois dans le corpus
JULES V A N VOLXEM apparait 7 fois dans le corpus
Denys apparait 7 fois dans le corpus
q u ' i apparait 6 fois dans le corpus
HERRY apparait 6 fois dans le corpus
Le Président apparait 5 fois dans le corpus
HELLEBAUT apparait 5 fois dans le corpus
Budget apparait 5 fois dans le corpus
Bockstael apparait 5 fois dans le corpus
Medaets apparait 4 fois dans le corpus
M. Limauge apparait 4 fois dans le corpus
Secrétaire apparait 4 fois dans le corpus
M. VAN DEN KERCKHOVE apparait 4 fois dans le corpus
sieur Marinus apparait 4 fois dans le corpus
Président apparait 4 fois dans le corpus
Herry apparait 4 fois